In [4]:
from darts.models import NLinearModel
from darts import TimeSeries
import json
import pandas as pd
from transformers import BertTokenizer, BertModel, AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn
import numpy as np

In [5]:
"""
input: x_time and x_text

# LLM 2 ways
x_time: ("110.23") --> llm tokenizer --> time_embd_for_llm
x_text: input_id

# NLinear
x_time --> n linear --> time_embd_for_nlinear
x_text --> bert or llm embd --> text_embd for nlinear
"""

'\ninput: x_time and x_text\n\n# LLM 2 ways\nx_time: ("110.23") --> llm tokenizer --> time_embd_for_llm\nx_text: input_id\n\n# NLinear\nx_time --> n linear --> time_embd_for_nlinear\nx_text --> bert or llm embd --> text_embd for nlinear\n'

In [6]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)


text="""
EXTENDED FORECAST DISCUSSION
NWS WEATHER PREDICTION CENTER COLLEGE PARK MD
1042 AM EDT FRI APR 04 2014

VALID 12Z MON APR 07 2014 - 12Z FRI APR 11 2014

...PATTERN OVERVIEW AND GUIDANCE/UNCERTAINTY ASSESSMENT...

A RATHER ACTIVE AND MORE SPRINGTIME UPPER PATTERN WILL CONTINUE
OVER THE CONUS THROUGH NEXT WEEK. AMPLIFIED RIDGE/TROUGH OVER THE
PAC NW/EASTERN THIRD OF THE COUNTRY... RESPECTIVELY... WILL RELAX
IN TIME AS PACIFIC ENERGY ENCROACHES UPON CALIFORNIA BY THE END OF
NEXT WEEK. THE MODELS/ENSEMBLES ARE IN GOOD AGREEMENT THROUGHOUT
THE FORECAST WITH MAINLY TIMING DIFFERENCES AMONG THE GFS/GEFS
MEMBERS AND ECMWF/EC ENSEMBLE MEMBERS CAMPS. THE GEFS MEMBERS WERE
MOSTLY QUICKER WITH THE FLOW THAN THE REST OF THE GUIDANCE...AND
WERE NOT PREFERRED CONSIDERING LARGE SCALE PATTERN AMPLITUDE
TREND.  IN PARTICULAR BY LATER NEXT WEEK... THE ENERGY APPROACHING
OR MOVING INTO CALIFORNIA HAS SHOWN A FAIR AMOUNT OF SPREAD AMONG
THE ENSEMBLES. THE GEFS AGAIN IS QUICKEST AND NOT PREFERRED IN LUE
OF SLOWER ECMWF ENSEMBLES.  THE 12 UTC ECMWF FROM YESTERDAY SEEMS
THE CLOSEST DETERMINISTIC MODEL MATCH WITH THIS SOLUTION AND
CONSIDERING THE CLOSED NATURE OF THE SYSTEM AT THIS LATITUDE IT
VERY WELL MAY END UP BEING EVEN LESS PROGRESSIVE.  THE LATEST 00
UTC ECMWF AND YESTERDAYS 00 UTC ECMWF INSTEAD SEEM TO OFFER KINDA
FUNKY LOOKING SYSTEM PHASING BY DAYS 6/7 OVER THE ERN PACIFIC THAT
SEEMS LESS LIKELY TO OCCUR AND WERE DISCOUNTED GIVEN THE LONG TIME
FRAME.

WPC MEDIUM RANGE SURFACE FRONTS/PRESSURES AND 500 MB PROGS ALONG
WITH ASSOCIATED SENSIBLE WEATHER GRIDS HAVE BEEN PRIMARILY DERIVED
FROM OVERNIGHT WPC CONTIUNITY THAT WAS ORIGINALLY BASED UPON A
BLEND OF YESTERDAYS 12 UTC ECMWF AND INCREASINGLY OVER TIME ECMWF
ENSEMBLES.  A MANUAL ADJUSTMENT WAS APPLIED TO SLIGHTLY SLOW LATER
WEEK SYSTEM APPROACH TO CA GIVEN REASONINGS ABOVE. 
OVERALL...CONFIDENCE REMAINS NEAR TO ABOVE AVERAGE THIS FORECAST
PERIOD.

...SENSIBLE WEATHER HIGHLIGHTS...

LEAD EASTERN SYSTEM WILL SPREAD A LARGE AREA OF MODEST RAINS EAST
OF 90W... WITH THE BEST CHANCE FOR HEAVIER RAIN IN THE SOUTHEAST.
BELOW AVERAGE TEMPS ARE EXPECTED UNTIL THE TROUGH AXIS PASSES
OFFSHORE LATE WED/D5... DOWN THROUGH FLORIDA AS THE FRONT PUSHES
WELL TO THE SOUTH. THE WEST WILL SEE AN EXPANDING AREA OF ABOVE
AVERAGE TEMPERATURES PUSH EASTWARD PAST THE ROCKIES AS HEIGHTS
RISE THROUGH THE CENTRAL STATES. MODEST WARMING WILL PUSH INTO THE
EASTERN CONUS LATER IN THE WEEK. PRECIPITATION NEXT THU-FRI SHOULD
BE MINIMAL COMPARED TO THE RECENT HEAVY RAIN EVENTS... LIMITED TO
THE NORTHERN TIER AS A SYSTEM PUSHES THROUGH SOUTHERN CANADA AND
ALSO INTO SOME PORTIONS OF CALIFORNIA AS THE ENERGY NEARS THE
COAST.

SCHICHTEL
"""

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
df = pd.read_csv('text_data.csv')

In [20]:
df["Text"].values

array(['EXTENDED FORECAST DISCUSSION\nNWS WEATHER PREDICTION CENTER COLLEGE PARK MD\n1042 AM EDT FRI APR 04 2014\n\nVALID 12Z MON APR 07 2014 - 12Z FRI APR 11 2014\n\n...PATTERN OVERVIEW AND GUIDANCE/UNCERTAINTY ASSESSMENT...\n\nA RATHER ACTIVE AND MORE SPRINGTIME UPPER PATTERN WILL CONTINUE\nOVER THE CONUS THROUGH NEXT WEEK. AMPLIFIED RIDGE/TROUGH OVER THE\nPAC NW/EASTERN THIRD OF THE COUNTRY... RESPECTIVELY... WILL RELAX\nIN TIME AS PACIFIC ENERGY ENCROACHES UPON CALIFORNIA BY THE END OF\nNEXT WEEK. THE MODELS/ENSEMBLES ARE IN GOOD AGREEMENT THROUGHOUT\nTHE FORECAST WITH MAINLY TIMING DIFFERENCES AMONG THE GFS/GEFS\nMEMBERS AND ECMWF/EC ENSEMBLE MEMBERS CAMPS. THE GEFS MEMBERS WERE\nMOSTLY QUICKER WITH THE FLOW THAN THE REST OF THE GUIDANCE...AND\nWERE NOT PREFERRED CONSIDERING LARGE SCALE PATTERN AMPLITUDE\nTREND.  IN PARTICULAR BY LATER NEXT WEEK... THE ENERGY APPROACHING\nOR MOVING INTO CALIFORNIA HAS SHOWN A FAIR AMOUNT OF SPREAD AMONG\nTHE ENSEMBLES. THE GEFS AGAIN IS QUICKEST A

In [29]:
tokens = [len(tokenizer.encode(t)) for t in df["Text"].values]

In [30]:
sum(tokens) / len(tokens)

1176.234060640408

In [3]:
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model = BertModel.from_pretrained('bert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

# device = "cuda:2"
# # llm = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)

# # embeddings
# embd_dim = 4096
# window_size = 5
# embed_timeseries = nn.Linear(window_size, embd_dim)

# # Initialize weights and bias
# nn.init.normal_(embed_timeseries.weight, mean=0.0, std=0.01)
# nn.init.zeros_(embed_timeseries.bias)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/4 [00:11<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU  has a total capacity of 39.38 GiB of which 162.81 MiB is free. Process 383810 has 30.32 GiB memory in use. Process 360939 has 8.86 GiB memory in use. Of the allocated memory 8.46 GiB is allocated by PyTorch, and 1.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
# run Upload_finance.ipynb
# on path: "/data/kai/forecasting/data/summary/summary_with_price_v0.2/AMD"

data_path = "/data/kai/forecasting/data/test/AMD.csv"
df = pd.read_csv(data_path)

# n = df.shape[0]
# train_n = int(n * 0.8)
# train_df = df.iloc[:train_n]
# valid_df = df.iloc[train_n:]

def extract_inputs(df):
    time_series = df["input"].apply(lambda x: json.loads(x)["share_price"]).values
    input_time = np.array([time_series[i: i+window_size] for i in range(len(df) - window_size*2)])
    output_time = np.array([time_series[i: i+window_size] for i in range(window_size, len(df) - window_size)])
    # TODO: change to all instead of "summary"
    input_text = df["input"].apply(lambda x: json.loads(x)["summary"]).values.tolist()
    input_text = [input_text[i:i+window_size] for i in range(len(df) - window_size*2)]
    output_text = [input_text[i:i+window_size] for i in range(window_size, len(df)-window_size)]
    
    # input_text: N x T x len(str)
    return input_time, output_time, input_text, output_text

input_time, output_time, input_text, output_text = extract_inputs(df)
print("Input time:", len(input_time), len(input_time[0]), input_time[0])
print("Output time:", len(output_time), len(output_time[0]), output_time[0])
print("Input text:", len(input_text), len(input_text[0]), len(input_text[0][0]))
print("Output text:", len(output_text), len(output_text[0]), len(output_text[0][0]))

Input time: 267 5 [111.98 108.41 102.95 105.53 106.46]
Output time: 267 5 [104.29 102.25 109.33 115.37 111.69]
Input text: 267 5 159
Output text: 267 5 5


In [54]:
# 1) Use linear embedding

# B x window x 1
time_input = torch.tensor(input_time[0]).unsqueeze(0).unsqueeze(-1).type(torch.float32)
input_embeds_time = embed_timeseries(time_input).to(device)

# hidden_states = inputs_embeds
input_text_formatted = ','.join(input_text[0])
embeddings = tokenizer(input_text_formatted, padding=True, truncation=True, return_tensors="pt")["input_ids"]
input_embeds_text = llm.base_model.embed_tokens(embeddings[:window_size].to(device))

# concatenate
llm_embed = torch.cat((input_embeds_time, input_embeds_text), dim=1)
print(input_embeds_time.shape, input_embeds_text.shape, llm_embed.shape)

# or add embedding for each token
# tokenized[ day_1+time_embd_day_1, day_2+time_embd_day_1, day_3+time_embd_day_1...]


# or add some exponential weighting
# tokenized[ day_1+time_embd*[||___], day_2+time_embd_day_1*[__||___], day_3+time_embd_day_1*[____||]...]


# how to modify llama embedding
# transformers / models / llama / modeling_llama.py
# Class LlamaModel: def forward
# inputs_embeds
# line 945: hidden_states = inputs_embeds

torch.Size([1, 5, 4096]) torch.Size([1, 632, 4096]) torch.Size([1, 637, 4096])
